project/
├── images/
│   ├── train/
│   └── val/
├── labels/
│   ├── train/
│   └── val/
└── data.yaml

In [2]:
import os
import xml.etree.ElementTree as ET
import cv2
import matplotlib.pyplot as plt

In [19]:
xml_dir = '/mask_recognition/data/dataset_kaggle_Face_Mask_Detection/annotations'
label_dir = '/mask_recognition/data/dataset_kaggle_Face_Mask_Detection/labels'
os.makedirs(label_dir, exist_ok=True)

classes = {
    "with_mask": 0,
    "without_mask": 1,
    "mask_weared_incorrect": 2
}

for xml_file in os.listdir(xml_dir):
    if os.path.splitext(xml_file)[1] != '.xml':
        continue

    tree = ET.parse(os.path.join(xml_dir, xml_file))
    root = tree.getroot()

    w = int(root.find("size/width").text)
    h = int(root.find("size/height").text)

    yolo_lines = []

    for obj in root.findall("object"):
        class_name = obj.find("name").text
        if class_name not in classes:
            continue

        class_id = classes[class_name]

        bbox = obj.find("bndbox")
        xmin = int(bbox.find("xmin").text)
        ymin = int(bbox.find("ymin").text)
        xmax = int(bbox.find("xmax").text)
        ymax = int(bbox.find("ymax").text)

        cx = (xmin + xmax) / 2 / w
        cy = (ymin + ymax) / 2 / h
        bw = (xmax - xmin) / w
        bh = (ymax - ymin) / w
        
        yolo_lines.append(f"{class_id} {cx:.6f} {cy:.6f} {bw:.6f} {bh:.6f}")

    filename = os.path.splitext(xml_file)[0]
    with open(os.path.join(label_dir, f"{filename}.txt"), "w") as f:
        f.write("\n".join(yolo_lines))

In [ ]:
img_root = '/mask_recognition/data/dataset_kaggle_Face_Mask_Detection/images'
label_root = '/mask_recognition/data/dataset_kaggle_Face_Mask_Detection/labels'

for index in range(50):
    img_path = os.path.join(img_root, f"maksssksksss{index}.png")
    label_path = os.path.join(label_root, f"maksssksksss{index}.txt")

    img = cv2.imread(img_path)

    h, w, _ = img.shape

    with open(label_path) as f:
        for line in f.readlines():
            label, cx, cy, bx, by = [float(value) for value in line.split()]

            x1 = int((cx - bx / 2) * w)
            x2 = int((cx + bx / 2) * w)
            y1 = int((cy - by / 2) * h)
            y2 = int((cy + by / 2) * h)

            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 3)

    plt.imshow(img)
    plt.show()
        